## Fake NEws Classificer USsing Bidirectional LSTM

In [1]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd



In [4]:
df=pd.read_csv('train.csv',engine='python',on_bad_lines='skip')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

,0
id,0
title,269
author,934
text,21
label,0


In [9]:
df.shape

(8680, 5)

In [8]:
df=df.dropna()

In [13]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
df = df.reset_index(drop=True)

In [14]:
## get the independent features
X=df.drop('label',axis=1)

In [15]:
y=df['label']

In [16]:
## Check whether dataset is balanced or not
y.value_counts()

,count
label,
0,4935
1,3745


In [17]:
import tensorflow as tf

In [18]:
tf.__version__

'2.17.1'

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [20]:
## vocabulary size
voc_size=5000

In [26]:

messages=X.copy()

In [22]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [23]:
import nltk
import re
from nltk.corpus import stopwords

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
messages.reset_index(inplace=True)

In [28]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [29]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [30]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[252, 394, 2738, 2002, 1027, 4975, 2691, 2861, 2887, 1054],
 [3183, 2525, 4463, 2195, 1434, 4340, 4995],
 [4403, 1075, 3010, 1919],
 [3841, 1076, 305, 235, 3250, 412],
 [3992, 1434, 1043, 520, 1401, 2072, 1434, 3827, 3179, 356],
 [3272,
  2767,
  2105,
  198,
  4382,
  4255,
  3138,
  1207,
  4666,
  3715,
  1422,
  3495,
  3219,
  4650,
  4995],
 [1094, 4526, 453, 95, 959, 4820, 4347, 355, 4079, 4719, 1906],
 [110, 4447, 4718, 2176, 2827, 4467, 4255, 2082, 4079, 4719, 1906],
 [4718, 3811, 475, 3430, 620, 4046, 2068, 2486, 4255, 737],
 [3610, 4631, 1296, 1413, 4445, 285, 1901, 3853],
 [4606, 4824, 1343, 362, 2861, 1783, 4958, 566, 2931, 3511, 2838],
 [235, 4114, 1027, 4046, 4255, 2827],
 [2984, 3553, 4852, 1360, 3896, 2247, 3741, 1685, 1918],
 [4401, 3409, 4792, 2479, 3751, 1656, 3492, 4079, 4719, 1906],
 [276, 2274, 2011, 3231, 2928, 4079, 4719, 1906],
 [2588, 2840, 716, 2185, 3305, 3705, 4330, 122, 2682, 1008],
 [3807, 893, 2525],
 [4152, 107, 277, 4962, 4255, 2216, 1204, 4995],
 [3

## Embedding Representation

In [31]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 2861, 2887, 1054],
       [   0,    0,    0, ..., 1434, 4340, 4995],
       [   0,    0,    0, ..., 1075, 3010, 1919],
       ...,
       [   0,    0,    0, ..., 4079, 4719, 1906],
       [   0,    0,    0, ..., 4079, 4719, 1906],
       [   0,    0,    0, ..., 1695, 3522, 3852]], dtype=int32)

In [33]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_shape=(sent_length,)))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 200)                 │         112,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

None


In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [38]:
X_train.shape

(5815, 20)

In [37]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=32)

Epoch 1/15
182/182 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - accuracy: 0.9974 - loss: 0.0073 - val_accuracy: 0.9026 - val_loss: 0.6931
Epoch 2/15
182/182 ━━━━━━━━━━━━━━━━━━━━ 15s 36ms/step - accuracy: 0.9975 - loss: 0.0097 - val_accuracy: 0.8883 - val_loss: 0.6471
Epoch 3/15
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - accuracy: 0.9996 - loss: 0.0015 - val_accuracy: 0.8939 - val_loss: 0.7410
Epoch 4/15
182/182 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 1.0000 - loss: 2.1153e-04 - val_accuracy: 0.8914 - val_loss: 0.7834
Epoch 5/15
182/182 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.9995 - loss: 9.4822e-04 - val_accuracy: 0.8921 - val_loss: 0.8091
Epoch 6/15
182/182 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 1.0000 - loss: 8.0799e-05 - val_accuracy: 0.8939 - val_loss: 0.8350
Epoch 7/15
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - accuracy: 1.0000 - loss: 5.1689e-05 - val_accuracy: 0.8925 - val_loss: 0.8536
Epoch 8/15
182/182 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 1.0000 - l

In [39]:
y_pred=model.predict(X_test)

90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


In [40]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [41]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[1474,  162],
       [ 146, 1083]])

In [42]:
print(accuracy_score(y_test,y_pred))

0.8924956369982548
